In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from selenium import webdriver
import time
import string
from datetime import datetime
pd.options.display.float_format = '{:.0f}'.format

In [2]:
# BaseURL of Yahoo Finance website.
URL = "https://finance.yahoo.com/"

In [3]:
def Live_Data(cname):
    driver = webdriver.Chrome(executable_path = r"C:\Users\KIIT\Desktop\Web Driver\chromedriver.exe")
    print("Catching up with the site...")
    driver.get(URL)
    print("Searching the company")
    driver.find_element_by_xpath("//input[@placeholder = 'Search for news, symbols or companies']").send_keys(cname)
    driver.find_element_by_xpath("//button[@id= 'header-desktop-search-button']").click()
    print("Found the page, you requested!")
    # time.sleep(2)
    # Driver clicks on Historical Data tab and sleeps for 2 seconds.
    # driver.find_element_by_xpath("//span[text() = 'Summary']").click()
    # time.sleep(2)
    html = driver.execute_script('return document.body.innerHTML;')
    soup = BeautifulSoup(html,'lxml')
    end_loop = ['At close:  4:00PM EST','At close:  3:30PM IST']
    i = 1
    while(1):
        print(''.join([entry.text for entry in soup.find_all('div',{'id':"quote-market-notice"}
                            ,{'class':"C($tertiaryColor) D(b) Fz(12px) Fw(n) Mstart(0)--mobpsm Mt(6px)--mobpsm"})]))
        if(''.join([entry.text for entry in soup.find_all('div',{'id':"quote-market-notice"}
                            ,{'class':"C($tertiaryColor) D(b) Fz(12px) Fw(n) Mstart(0)--mobpsm Mt(6px)--mobpsm"})]) in end_loop):
            print("The Market for",[entry.text for entry in soup.find_all('h1', {'class':'D(ib) Fz(18px)'})],"is closed, right now!")
            print("Completed!")
            break
        elif(i==1):
            html = driver.execute_script('return document.body.innerHTML;')
            soup = BeautifulSoup(html,'lxml')
            print("Getting the data for",[entry.text for entry in soup.find_all('h1', {'class':'D(ib) Fz(18px)'})])
            Real_Time_Price = [entry.text for entry in soup.find_all('span', {'class':'Trsdu(0.3s) Fw(b) Fz(36px) Mb(-4px) D(ib)'})]
            temp = [entry.text for entry in soup.find_all('span', {'data-reactid': "33"})]
            temp = temp[1:]
            headers = [entry.text for entry in soup.find_all('td', {'class':'C($primaryColor) W(51%)'})]
            getters = [entry.text for entry in soup.find_all('td', {'class':'Ta(end) Fw(600) Lh(14px)'})]
            res = dict(zip(headers, getters))
            now = datetime.now()
            now = now.strftime("%d/%m/%Y %H:%M:%S")
            df_temp = pd.DataFrame(list(zip(Real_Time_Price,temp)),columns=['Real_Time_Price','Difference'])
            df_temp['Date'] = now
            df_temp2 = pd.DataFrame(res,index=[0])
            df_temp2 = df_temp2[['Previous Close','Open','Volume']]
            frames = [df_temp,df_temp2]
            df_final = pd.concat(frames,axis=1)
            df_final.set_index('Date',inplace=True)
            df_final.to_csv('file.csv')
            print("#############",i)
            time.sleep(10)
            i = i + 1 
        else:
            html = driver.execute_script('return document.body.innerHTML;')
            soup = BeautifulSoup(html,'lxml')
            Real_Time_Price = [entry.text for entry in soup.find_all('span', {'class':'Trsdu(0.3s) Fw(b) Fz(36px) Mb(-4px) D(ib)'})]
            temp = [entry.text for entry in soup.find_all('span', {'data-reactid': "33"})]
            temp = temp[1:]
            headers = [entry.text for entry in soup.find_all('td', {'class':'C($primaryColor) W(51%)'})]
            getters = [entry.text for entry in soup.find_all('td', {'class':'Ta(end) Fw(600) Lh(14px)'})]
            res = dict(zip(headers, getters))
            now = datetime.now()
            now = now.strftime("%d/%m/%Y %H:%M:%S")
            df1_temp = pd.DataFrame(list(zip(Real_Time_Price,temp)),columns=['Real_Time_Price','Difference'])
            df1_temp['Date'] = now
            df1_temp2 = pd.DataFrame(res,index=[0])
            df1_temp2 = df1_temp2[['Previous Close','Open','Volume']]
            frames = [df1_temp,df1_temp2]
            df1_final = pd.concat(frames,axis=1)
            df1_final.set_index('Date',inplace=True)
            frames = [df_final,df1_final]
            df = pd.concat(frames)
            df_final = df
            df_final.to_csv('file.csv')
            print("#############",i)
            time.sleep(10)
            i = i+1

In [5]:
Name = input("Write the full company name or ticker ").upper()
Live_Data(Name)

Write the full company name or ticker goog
Catching up with the site...
Searching the company
Found the page, you requested!
At close:  4:00PM EST
The Market for ['Alphabet Inc. (GOOG)'] is closed, right now!
Completed!


Name, Prev close, Change in values(%), Volume